In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr

# 数据集处理

本次数据分为 train.csv 和 test.csv。每个文件有10列，前9列为特征 （都为离散型），最后一列是标签（只有1和-1两种取值）。

## 数据集读取函数实现

In [2]:
def loadDataSet(filePath):
    ''' 数据集读取函数'''
    data, label = [], []
    # 读取数据集
    with open(filePath) as f:
        for line in f.readlines():
            temp = line.strip().split(",")
            data.append([float(i) for i in temp[:-1]])
            if temp[-1] != '?':
                temp[-1] = float(temp[-1])
            label.append(temp[-1])
    ##### 输出数据集相关信息 ##########
    print("data dimension of dataset：", len(data[0]))
    print("number of sample in data :", len(data))
    print("label frequency:", dict(Counter(label)))
    ##### 输出数据集相关信息 ##########
    return np.array(data), np.array(label)

## 读取原始训练集

In [3]:
trainSet_origin, trainSet_label_origin = loadDataSet('.\\data\\train.csv')
trainSet_origin

data dimension of dataset： 9
number of sample in data : 787
label frequency: {1.0: 323, -1.0: 464}


array([[ 23.,   1.,   2., ...,   2.,   0.,   0.],
       [ 36.,   1.,   1., ...,   2.,   3.,   0.],
       [ 33.,   1.,   3., ...,   2.,   3.,   0.],
       ..., 
       [ 49.,   0.,   1., ...,   1.,   1.,   1.],
       [ 21.,   1.,   3., ...,   2.,   1.,   0.],
       [ 31.,   3.,   3., ...,   1.,   3.,   0.]])

## 读取测试集

In [4]:
testSet, testSet_lable = loadDataSet('.\\data\\test.csv')
testSet

data dimension of dataset： 9
number of sample in data : 300
label frequency: {'?': 300}


array([[ 32.,   3.,   3., ...,   0.,   3.,   0.],
       [ 24.,   1.,   2., ...,   1.,   2.,   0.],
       [ 32.,   3.,   3., ...,   0.,   2.,   0.],
       ..., 
       [ 43.,   2.,   2., ...,   2.,   2.,   0.],
       [ 21.,   1.,   1., ...,   3.,   2.,   0.],
       [ 30.,   2.,   3., ...,   2.,   2.,   0.]])

## 从训练集中划分验证集

### 随机划分

典型的从训练集中划分验证集的方法是：划分训练集中的30%为验证集，划分过程采用随机选取的方式。

In [5]:
def splitTrainSet2ValidateSet(trainSet_, trainSet_label_, rate=0.3):
    '''从训练集中划分验证集'''
    #整合数据集和对应标签在同一个数组上，方便后续划分
    allData = np.column_stack((trainSet_, trainSet_label_))
    #随机打乱数据集
    np.random.shuffle(allData)
    #得到要划分的验证集的样本个数
    splitNum = int(allData.shape[0]*rate)
    #划分数据集
    validateSet_ = allData[:splitNum, :-1]
    validateSet_label_ = allData[:splitNum, -1]
    trainSet_new = allData[splitNum:, :-1]
    trainSet_label_new = allData[splitNum:, -1]
    return trainSet_new, trainSet_label_new, validateSet_, validateSet_label_

temp = splitTrainSet2ValidateSet(trainSet_origin, trainSet_label_origin)
trainSet, trainSet_label, validateSet, validateSet_label = temp
print('划分后的验证集和训练集的维度：', validateSet.shape, trainSet.shape)

划分后的验证集和训练集的维度： (236, 9) (551, 9)


### 手动划分

有一些同学采用手动划分的方式来划分数据集，为了更好地对比结果，这里也使用手动划分的方式来得到验证集。

具体划分方法为：在原始训练集的基础上取前100个和后100个样本作为验证集。

划分后验证集和训练集各自保存在`validation_1.csv`和`train_1.csv`中。

In [6]:
trainSet2, trainSet2_label = loadDataSet('.\\data\\train_1.csv')
trainSet2
validateSet2, validateSet2_label = loadDataSet('.\\data\\validation_1.csv')
validateSet2

data dimension of dataset： 9
number of sample in data : 587
label frequency: {1.0: 223, -1.0: 364}


array([[ 28.,   3.,   3., ...,   0.,   3.,   0.],
       [ 36.,   3.,   3., ...,   1.,   3.,   0.],
       [ 47.,   3.,   3., ...,   0.,   2.,   0.],
       ..., 
       [ 22.,   3.,   3., ...,   1.,   0.,   1.],
       [ 36.,   1.,   2., ...,   1.,   2.,   0.],
       [ 47.,   1.,   1., ...,   1.,   1.,   0.]])

data dimension of dataset： 9
number of sample in data : 200
label frequency: {1.0: 100, -1.0: 100}


array([[ 23.,   1.,   2., ...,   2.,   0.,   0.],
       [ 36.,   1.,   1., ...,   2.,   3.,   0.],
       [ 33.,   1.,   3., ...,   2.,   3.,   0.],
       ..., 
       [ 49.,   0.,   1., ...,   1.,   1.,   1.],
       [ 21.,   1.,   3., ...,   2.,   1.,   0.],
       [ 31.,   3.,   3., ...,   1.,   3.,   0.]])

# 特征选取

## 信息增益和信息增益率

### 实现原理

#### 信息增益

计算步骤如下：

- 1.计算数据集D的熵：$H(D)=-\sum_{d\in D}p(d)*log[p(d)]$

  - $p(d)$为标签d出现的概率


- 2.计算特征A相对于数据集D的条件熵：$H(D|A)=\sum_{a \in A}p(a)H(D|A=a)$
  
  - $H(D|A=a)$为特征A取值为a时D中对应的子数据集的熵


- 3.计算信息增益：$Gain(D,A)=H(D)-H(D|A)$


在选用信息增益来为决策树选取划分结点时，**信息增益最大**的特征将被视为当前的划分点。

#### 信息增益率

计算步骤如下：

- 1.计算特征A相对于数据集D的信息增益$Gain(D,A)$


- 2.计算特征A的熵：$H(A)=-\sum_{a\in A}p(a)*log[p(a)]$


- 3.计算信息增益率：$GainRatio(D,A)=Gain(D,A)/H(A)$


在选用信息增益率来为决策树选取划分结点时，**信息增益率最大**的特征将被视为当前的划分点。

### 算法实现

In [7]:
def calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=False):
    '''计算数据集每一列（特征）的信息增益或信息增益率'''
    def calcEntropy(data):
        '''计算单列数据的熵'''
        probs_ = np.array(list(Counter(data).values()))/data.shape[0]
        ans = -1*(probs_*np.log2(probs_)).sum()
        return ans
    #计算数据集的熵
    dataSetEntropy = calcEntropy(label)
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    infoGains = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        entropys = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集标签
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 子数据集标签 的 熵
            entropys[index] = calcEntropy(subLabel)
        #计算基于当前特征的 条件熵
        condEntropy = (probs*entropys).sum()
        #计算基于当前特征的 信息增益
        infoGains[featureId] = dataSetEntropy - condEntropy
        #若是计算 信息增益率，则要除以 当前特征的 熵 
        if calcInfoGainRate:
            denominator = calcEntropy(curFeature)
            #当 当前特征的 熵 为0时，信息增益也为0，此处避免除零错误
            if denominator != 0:
                infoGains[featureId] /= calcEntropy(curFeature)
    return infoGains

def calcInfoGain(dataSet, label):
    '''计算数据集每一列（特征）的信息增益'''
    return calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=False)

def calcInfoGainRate(dataSet, label):
    '''计算数据集每一列（特征）的信息增益率'''
    return calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=True)

### 测试信息增益计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [8]:
ansA = calcInfoGain(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansA
print("选取特征的下标（从0开始）为：", np.argmax(ansA) )

每个特征的计算值如下：


array([ 0.09219297,  0.00969743,  0.0206067 ,  0.10927331,  0.00058833,
        0.00121617,  0.00480776,  0.00819787,  0.01409991])

选取特征的下标（从0开始）为： 3


### 测试信息增益率计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [9]:
ansB = calcInfoGainRate(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansB
print("选取特征的下标（从0开始）为：", np.argmax(ansB) )

每个特征的计算值如下：


array([ 0.01876377,  0.00496779,  0.01309753,  0.03547937,  0.00103376,
        0.00150852,  0.0030534 ,  0.00445081,  0.03086695])

选取特征的下标（从0开始）为： 3


## 基尼指数

### 实现原理

计算公式为：$$Gini(D,A)=\sum_{a \in A}p(a)*gini(D|A=a)$$

其中，在特征A取值为a时对应的数据集D的子集的基尼指数为：
$$gini(D|A=a)=\sum_{i=1}^np_i(1-p_i)=1-\sum_{i=1}^np_i^2$$

在选用基尼指数来为决策树选取划分结点时，**基尼指数最小**的特征将被视为当前的划分点。

### 算法实现

In [10]:
def calcGiniIndex(dataSet, label):
    '''计算数据集每一列（特征）的Gini指数'''
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    giniIndexs = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        subGiniIndexs = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 每个取值下 数据集的 gini指数
            sub_values = np.array(list(Counter(subLabel).values()))
            sub_probs = sub_values / subLabel.shape[0]
            subGiniIndexs[index] = 1 - (sub_probs**2).sum()
        #计算基于当前特征的 gini 指数
        giniIndexs[featureId] = (probs*subGiniIndexs).sum()
    return giniIndexs

### 测试基尼指数计算结果的正确性

使用训练集进行测试，与其他同学的结果进行对比后，可知如下结果是正确的。

In [11]:
ansC = calcGiniIndex(trainSet, trainSet_label)
print("每个特征的计算值如下：")
ansC
print("选取特征的下标（从0开始）为：", np.argmax(ansC) )

每个特征的计算值如下：


array([ 0.43014533,  0.4774742 ,  0.47165246,  0.4254384 ,  0.48346148,
        0.48304835,  0.48065305,  0.47857447,  0.47512049])

选取特征的下标（从0开始）为： 4


## 特征选取类实现

为了更方便地使用3种特征选取，这里实现特征选取类。

In [12]:
class featureSelection:
    '''特征选取类：根据不同特征选取方法选取最优划分特征'''
    
    def __init__(self, method):
        self.method = method
        
    def getFeatureIndex(self, dataSet, label):
        '''得到最优划分属性的下标（从0开始）'''
        if self.method == 'ID3':
            return np.argmax(calcInfoGain(dataSet, label))
        elif self.method == 'C4.5':
            return np.argmax(calcInfoGainRate(dataSet, label))
        elif self.method == 'CART':
            return np.argmin(calcGiniIndex(dataSet, label))
        else:
            print("ERROR: method not define!")
        
##############测试程序###################
ID3 = featureSelection('ID3')
C45 = featureSelection('C4.5')
CART = featureSelection('CART')

a = ID3.getFeatureIndex(trainSet, trainSet_label)
b = C45.getFeatureIndex(trainSet, trainSet_label)
c = CART.getFeatureIndex(trainSet, trainSet_label)
[a,b,c]
##############测试程序###################

[3, 3, 3]

# 决策树算法

## 构建决策树的递归终止条件

假设当前结点的数据集为D，特征集为A，递归终止条件及对应的处理如下：

- 1.**D中的样本属于同一类别C，则将当前结点标记为C类叶结点**。
    - 最简单的一种递归终止条件，很好理解。


- 2.**A为空集，此时无法划分。将当前结点标记为叶结点，类别为D中出现最多的类**。
    - 比如数据集为`[['a'],['a'],['a'],['b']]`，标签为`['yes','no','yes','no']`，则经过一次划分后，特征集就为空集了。因此便无法划分下去了。而对应分类为`'a'`的标签有`['yes','no','yes']`，因此便需选择出现最多的类为结果。


- 3.**D中所有样本在A中所有特征上取值相同，此时无法划分。将当前结点标记为叶结点，类别为D中出现最多的类**。
    -  比如数据集为`[['a', 'b'],['a', 'b'],['a', 'b']]`，标签为`['yes','no','yes']`。此时数据集的两个特征的各自的取值都是一样的，此时也无法划分，因此便需选择出现最多的类为结果。
    
   
- 4.**D为空集，则将当前结点标记为叶结点，类别为父结点中出现最多的类**。
    - 这种情况在本次报告暂时还没有出现过，因此在实现中忽略该条件。


## 算法伪代码

```
buildTree(dataSet, label, featuresName)

输入数据：

-------------------------------------------------
dataSet：训练数据集
label：对应训数据集的标签
featuresName：训练数据集的特征名称向量
-------------------------------------------------

输出数据：

-------------------------------------------------
tree:构建好的决策树
-------------------------------------------------

算法流程：
-------------------------------------------------
1.若dataSet中的样本属于同一类别C，则将当前结点标记为C类叶结点。停止递归。
2.若dataSet为空集或dataSet中每个特征的取值都只有一个，则将当前结点标记为叶结点，类别为label中出现最多的类。停止递归。
3.选择最优划分属性。
4.以划分属性为结点构建一颗空树tree。
5.存储该结点对应的最可能出现的标签值（用于预测未知值），使用label出现最多的标签值。
6.对于划分属性的所有可能取值：
6.1.根据划分属性当前取值，得到划分后的子数据集subDataSet和子标签subLabel。
6.2.以buildTree(subDataSet, subLabel, featuresName)为分支结点。
7.返回决策树tree。
-------------------------------------------------
```


## 决策树类实现

In [13]:
class decisionTree:
    '''决策树类实现'''
    
    def __init__(self, method):
        self.featureSelectionMethod = featureSelection(method=method) #特征选取方法
    
    def __getBestSplitFeature(self, dataSet, label):
        '''得到数据集的最优划分属性的下标'''
        return self.featureSelectionMethod.getFeatureIndex(dataSet, label)
    
    def __getSubSet(self, dataSet, label, splitIndex, splitValue):
        '''根据划分属性的某个取值来得到对应的子数据集和标签'''
        #得到对应取值的样本下标
        sampleIndex = np.argwhere(dataSet[:, splitIndex]==splitValue)[:, 0]
        #得到在原数据集的基础上删除划分属性所在列对应的样本
        subDataSet = np.delete(dataSet, splitIndex, axis=1)[sampleIndex]
        #得到子标签
        subLabel = label[sampleIndex]
        return subDataSet, subLabel
    
    def __getMostCommonLabel(self, label):
        '''得到标签中出现最多次的数据'''
        return Counter(label).most_common(1)[0][0]
    
    def __buildTree(self, dataSet, label, featuresName):
        '''递归构建决策树'''
        #递归终止条件1：若数据都属于一个类别，则返回该类别
        if len(Counter(label)) == 1:
            return label[0]
        #递归终止条件2：若遍历完所有属性，则返回标签中出现次数最多的
        if len(dataSet) == 0:
            return self.__getMostCommonLabel(label)
        #递归终止条件3：若所有样本在所有特征上取值相同，则返回标签中出现次数最多的
        check = []
        ## 遍历所有特征，得到每个特征的取值的次数
        for featureIndex in range(dataSet.shape[1]):
            check.append(len(Counter(dataSet[:, featureIndex])))
        ## 若所有特征的取值都只有一个
        if Counter(check).get(1,-1)==len(check):
            return self.__getMostCommonLabel(label)
        
        #得到划分属性下标
        splitIndex = self.__getBestSplitFeature(dataSet, label)
        #得到划分属性
        splitFeature = featuresName[splitIndex]
        del(featuresName[splitIndex])
        #以划分属性为结点构建一颗空树
        tree = {splitFeature:{'tree':{}}} 
        #存储该结点对应的最可能出现的标签值（用于预测未知值）
        tree[splitFeature]['defaultLabel'] = self.__getMostCommonLabel(label)
        #遍历划分属性的所有可能取值
        for val in set(dataSet[:, splitIndex]):
            subDataSet, subLabel = self.__getSubSet(dataSet, label, splitIndex, val)
            tree[splitFeature]['tree'][val] = self.__buildTree(subDataSet, subLabel, 
                                                             featuresName[:])
        return tree
        
    def buildTree(self, dataSet, label, featuresName):
        '''得到决策树'''
        self.tree = self.__buildTree(dataSet, label, featuresName)
    
    def __apply(self, tree, sample, featuresName):
        '''递归应用构建好的决策树对数据集进行分类'''
        #得到根结点属性
        rootFeature = list(tree.keys())[0] 
        #对应根结点的树
        rootTree = tree[rootFeature]['tree']
        #根据结点名称找到结点对应的下标
        rootIndex = featuresName.index(rootFeature)
        #遍历树的所有可能的分支
        for val in rootTree.keys():
            if sample[rootIndex] == val:
                subTree = rootTree[val]
                #若接下来是一棵树
                if isinstance(subTree, dict):
                    return self.__apply(subTree, sample, featuresName)
                #若接下来是一个结点
                else:
                    return subTree
        #若出现未知值，则返回默认的标签值
        return tree[rootFeature]['defaultLabel']  
            
    def apply(self, dataSet, featuresName):
        '''对数据集进行分类'''
        ansLabel = np.zeros(dataSet.shape[0])
        #遍历测试数据集的每一个样本
        for index, sample in enumerate(dataSet):
            ansLabel[index] = self.__apply(self.tree, sample, featuresName)
        return ansLabel
        
    def getTree(self):
        '''返回训练好的以字典形式存储的决策树'''
        return self.tree

## 运行决策树

In [14]:
def runDecisionTree(trainSet_, trainSet_label_, validateSet_, validateSet_label_):
    '''运行决策树函数'''
    def run(method):
        test = decisionTree(method)
        featuresName_ = list(range(trainSet_.shape[1]))
        test.buildTree(trainSet_, trainSet_label_, featuresName_[:])
        ans = test.apply(validateSet_, featuresName_[:])
        diff = np.argwhere(ans == validateSet_label_)
        accur = 100*float(diff.shape[0]/validateSet_label_.shape[0])
        print(method+":", "%.3f%%" % accur)
        
    for method_ in ['ID3', 'C4.5', 'CART']:
        run(method_)

### 在原始训练集上运行决策树

理论上来说，在原始数据集上运行决策树理应得到的准确率为100%，但数据集中可能存在着干扰，即是一些本来应该划分到同一个标签的具有高相似的数据的训练标签可能不一致，因此准确率会接近100%而有时不会等于100%。

In [15]:
runDecisionTree(trainSet_origin, trainSet_label_origin, 
                                        trainSet_origin, trainSet_label_origin)

ID3: 100.000%
C4.5: 100.000%
CART: 100.000%


### 在随机划分的训练集和验证集上运行决策树

In [16]:
runDecisionTree(trainSet, trainSet_label, validateSet, validateSet_label)

ID3: 65.254%
C4.5: 64.407%
CART: 65.254%


### 在手动划分的训练集和验证集上运行决策树

In [17]:
runDecisionTree(trainSet2, trainSet2_label, validateSet2, validateSet2_label)

ID3: 58.500%
C4.5: 61.500%
CART: 58.500%


比较两种划分原始训练集的方式，随机划分得到的3种决策树的平均准确率要高于手动划分的。

可能的原因应是：随机划分的训练集和验证集中的标签分布较为相似或接近，而手动划分的则没有这个效果。

## 在测试集上应用决策树

综合上面3种决策树的表现，这里选择使用随机划分数据得到的CART决策树来对测试集进行预测。

### 得到最终预测结果

In [18]:
testTree = decisionTree('CART')
featuresName_ = list(range(trainSet.shape[1]))
testTree.buildTree(trainSet, trainSet_label, featuresName_[:])
resLabel = testTree.apply(testSet, featuresName_[:])

Counter(resLabel) #得到预测结果中标签的分布情况

Counter({-1.0: 186, 1.0: 114})

### 保存结果

In [19]:
with open('15352220_linzecheng.txt', 'w', encoding='utf-8') as f:
    for i in resLabel:
        _ = f.write(str(int(i))+'\n')

# 思考题

- 决策树有哪些避免过拟合的方法？
    - 剪枝，比如预剪枝和后剪枝
    - 使用多颗决策树，比如使用随机森林算法
    - 其他方法：
        - 限制树的深度
        - 设置一个节点能被继续拆分出子节点所需要的包含最少的样本个数
        - 设置最底层节点所需要包含的最少样本个数
    
    
- C4.5相比于ID3的优点是什么？
    - **ID3倾向于选择分支较多的属性**，而在大多数情况下，分支多的属性不一样是最有用的，比如数据集中每个样本都有一个唯一的id属性，若根据id属性进行划分，生成的树会显示非常庞大而低效。**C4.5在ID3的基础上除以特征的熵，这样就可以避免这个问题了**。这也就是C4.5的优点所在。
    
    
- 如何用决策树来判断特征的重要性？
    - 从直观上来讲，**判断一个属性重要不重要等价于判断其值的改变对结果的影响程度大不大**。
    - 假设特征A在一颗决策树D上的结点的次数为M，在此次实验范围内，M为1。那么对于每个使用特征A划分出新分支的结点N，**可计算一种评测指标（比如说Gini指数）在该结点的变化值（用该结点的Gini指数和所有分支的Gini指数的总和的差来代表）来代表特征A在结点的重要性**。而将这M个值求平均就可得到特征A对整颗决策树的重要性了。
    - 更进一步地，也可对多颗决策树来求平均得到特征对所有决策树的重要性。

# 友情测试

帮舍友测试下他手动划分的训练集和验证集。

In [20]:
trainSet3, trainSet3_label = loadDataSet('.\\data\\train_2.csv')
trainSet3
validateSet3, validateSet3_label = loadDataSet('.\\data\\validation_2.csv')
validateSet3
runDecisionTree(trainSet3, trainSet3_label, validateSet3, validateSet3_label)

data dimension of dataset： 9
number of sample in data : 590
label frequency: {1.0: 225, -1.0: 365}


array([[ 35.,   1.,   3., ...,   2.,   3.,   0.],
       [ 24.,   1.,   2., ...,   2.,   2.,   0.],
       [ 28.,   3.,   3., ...,   0.,   3.,   0.],
       ..., 
       [ 36.,   1.,   2., ...,   1.,   2.,   0.],
       [ 47.,   1.,   1., ...,   1.,   1.,   0.],
       [ 30.,   1.,   2., ...,   1.,   0.,   0.]])

data dimension of dataset： 9
number of sample in data : 197
label frequency: {1.0: 98, -1.0: 99}


array([[ 23.,   1.,   2., ...,   2.,   0.,   0.],
       [ 36.,   1.,   1., ...,   2.,   3.,   0.],
       [ 33.,   1.,   3., ...,   2.,   3.,   0.],
       ..., 
       [ 26.,   3.,   3., ...,   2.,   2.,   0.],
       [ 31.,   2.,   2., ...,   2.,   0.,   1.],
       [ 32.,   0.,   2., ...,   2.,   2.,   0.]])

ID3: 58.376%
C4.5: 60.914%
CART: 58.376%
